In [1]:
import json
import numpy as np
from scipy import stats
from datetime import datetime

import analitico
import analitico.plugin
import s24.plugin
from analitico.pandas import *
from analitico.schema import apply_schema

# pass api token to create sdk
sdk = analitico.authorize("tok_s24_579E5hOWw7k8")
print(datetime.utcnow())

2019-05-29 14:18:43.562128


In [2]:
# load table that is already a result of joining and processing several tables
df1 = sdk.get_dataframe("ds_s24_order")
df1.sample(n=10)

,id,store_id,customer_id,courier_id,picker_id,fulfillment_type,status,deliver_at_start,deliver_at_end,amount,...,deleted,gdo,vertical,created_at,updated_at,assigned_at,picked_at,paid_at,delivered_at,deleted_at
446641,914296,896402,896438,106027.0,nan,1,600,2018-12-22 10:00:00,2018-12-22 11:00:00,60.38,...,0,3,0,2018-12-21 18:30:18,2018-12-23 01:00:30,2018-12-21 21:53:33,nan,2018-12-22 10:01:36,2018-12-22 10:13:07,nan
245643,511621,491791,491827,58312.0,6.0,3,600,2018-03-13 19:00:00,2018-03-13 20:00:00,23.29,...,0,0,0,2018-03-13 17:47:02,2018-10-03 14:37:06,2018-03-13 17:50:58,2018-03-13 17:47:03,2018-03-13 20:04:16,2018-03-13 20:05:10,nan
189807,399949,379989,380025,84539.0,nan,1,600,2017-11-25 14:00:00,2017-11-25 15:00:00,64.12,...,0,1,0,2017-11-24 21:19:43,2018-10-03 15:28:55,2017-11-24 21:21:58,nan,2017-11-25 14:59:20,2017-11-25 15:23:52,nan
465056,951132,933402,933438,152873.0,nan,1,600,2019-01-13 16:00:00,2019-01-13 17:00:00,54.77,...,0,3,0,2019-01-13 13:37:24,2019-01-13 17:17:19,2019-01-13 14:04:24,nan,2019-01-13 16:53:37,2019-01-13 17:17:19,nan
587987,1197054,1180254,1180290,61472.0,nan,1,600,2019-05-04 11:00:00,2019-05-04 12:00:00,51.41,...,0,3,0,2019-05-03 16:21:15,2019-05-04 11:15:11,2019-05-04 08:51:38,nan,2019-05-04 10:44:02,2019-05-04 11:15:11,nan
184381,389097,369137,369173,88264.0,nan,1,600,2017-11-14 11:00:00,2017-11-14 12:00:00,21.31,...,0,3,0,2017-11-13 16:29:53,2018-10-03 15:28:35,2017-11-13 16:33:17,nan,2017-11-14 10:59:23,2017-11-14 11:57:50,nan
464954,950928,933196,933232,111161.0,nan,1,600,2019-01-14 18:00:00,2019-01-14 19:00:00,54.60,...,0,3,0,2019-01-13 11:33:37,2019-01-14 18:44:07,2019-01-14 15:52:10,nan,2019-01-14 17:37:04,2019-01-14 18:44:07,nan
73754,167612,147648,147656,42150.0,nan,1,600,2016-10-07 12:00:00,2016-10-07 13:00:00,68.21,...,0,0,0,2016-10-07 10:28:11,2018-10-03 15:40:31,2016-10-07 10:34:51,nan,2016-10-07 12:21:47,2016-10-07 12:33:36,nan
99317,218828,198864,198872,71782.0,nan,1,600,2017-02-13 14:00:00,2017-02-13 15:00:00,14.20,...,0,0,0,2017-02-13 12:59:03,2018-10-03 15:24:16,2017-02-13 13:05:26,nan,2017-02-13 14:00:58,2017-02-13 14:37:20,nan
214497,449329,429369,429405,109135.0,nan,3,600,2018-01-17 18:00:00,2018-01-17 19:00:00,54.68,...,0,2,0,2018-01-17 14:35:38,2018-10-03 14:35:35,2018-01-17 16:23:30,nan,2018-01-17 17:45:06,2018-01-17 18:07:42,nan


In [3]:
df1 = df1.copy()
df2 = augment_dates(df1, column="deliver_at_start")
df2

/Users/gionata/github/analitico/notebooks/analitico/pandas.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[column + ".year"] = dates.year.astype("category")


,id,store_id,customer_id,courier_id,picker_id,fulfillment_type,status,deliver_at_start.dayofweek,deliver_at_start.year,deliver_at_start.month,...,deleted,gdo,vertical,created_at,updated_at,assigned_at,picked_at,paid_at,delivered_at,deleted_at
0,20000,2,2,4302.0,nan,1,600,3,2015,5,...,0,0,0,2015-05-14 12:10:29,2018-10-03 15:38:03,2015-05-14 12:27:06,nan,2015-05-14 13:06:45,2015-05-14 13:07:24,nan
1,20002,4,4,4302.0,nan,1,600,3,2015,5,...,0,0,0,2015-05-14 12:16:29,2018-10-03 15:38:03,2015-05-14 12:24:03,nan,2015-05-14 13:28:44,2015-05-14 13:35:48,nan
2,20004,6,6,4302.0,nan,1,600,3,2015,5,...,0,0,0,2015-05-14 12:25:15,2018-10-03 15:38:03,2015-05-14 13:40:09,nan,2015-05-14 13:51:43,2015-05-14 14:52:09,nan
3,20006,8,8,4302.0,nan,1,600,3,2015,5,...,0,0,0,2015-05-14 12:26:36,2018-10-03 15:38:03,2015-05-14 13:52:22,nan,2015-05-14 14:24:50,2015-05-14 14:51:24,nan
4,20008,10,10,4302.0,nan,1,600,3,2015,5,...,0,0,0,2015-05-14 17:50:46,2018-10-03 15:38:03,2015-05-14 18:00:00,nan,2015-05-14 18:42:23,2015-05-14 18:58:45,nan
5,20010,12,12,12029.0,nan,1,600,3,2015,5,...,0,0,0,2015-05-14 17:53:46,2018-10-03 15:38:03,2015-05-14 18:43:15,nan,2015-05-14 20:13:06,2015-05-14 20:19:27,nan
6,20012,14,14,3081.0,nan,1,600,4,2015,5,...,0,0,0,2015-05-15 12:51:26,2018-10-03 15:38:03,2015-05-15 13:21:29,nan,2015-05-15 14:07:18,2015-05-15 14:28:20,nan
7,20014,16,16,nan,nan,1,200,4,2015,5,...,1,0,0,2015-05-15 13:17:49,2015-10-07 17:11:52,nan,nan,nan,nan,2015-05-15 14:06:21
8,20016,18,18,3777.0,nan,1,600,4,2015,5,...,0,0,0,2015-05-15 19:30:16,2018-10-03 15:38:03,2015-05-15 19:45:03,nan,2015-05-15 20:53:19,2015-05-15 21:04:08,nan
9,20018,20,20,12068.0,nan,1,300,5,2015,5,...,1,0,0,2015-05-16 14:49:45,2015-07-08 11:18:16,2015-05-16 16:08:45,nan,nan,nan,2015-05-16 17:21:43


In [4]:
dates = pd.DatetimeIndex(df2["updated_at"])
df2.loc[:,"updated_at.year"] = dates.year.astype("category")
df2

,id,store_id,customer_id,courier_id,picker_id,fulfillment_type,status,deliver_at_start.dayofweek,deliver_at_start.year,deliver_at_start.month,...,gdo,vertical,created_at,updated_at,assigned_at,picked_at,paid_at,delivered_at,deleted_at,updated_at.year
0,20000,2,2,4302.0,nan,1,600,3,2015,5,...,0,0,2015-05-14 12:10:29,2018-10-03 15:38:03,2015-05-14 12:27:06,nan,2015-05-14 13:06:45,2015-05-14 13:07:24,nan,2018
1,20002,4,4,4302.0,nan,1,600,3,2015,5,...,0,0,2015-05-14 12:16:29,2018-10-03 15:38:03,2015-05-14 12:24:03,nan,2015-05-14 13:28:44,2015-05-14 13:35:48,nan,2018
2,20004,6,6,4302.0,nan,1,600,3,2015,5,...,0,0,2015-05-14 12:25:15,2018-10-03 15:38:03,2015-05-14 13:40:09,nan,2015-05-14 13:51:43,2015-05-14 14:52:09,nan,2018
3,20006,8,8,4302.0,nan,1,600,3,2015,5,...,0,0,2015-05-14 12:26:36,2018-10-03 15:38:03,2015-05-14 13:52:22,nan,2015-05-14 14:24:50,2015-05-14 14:51:24,nan,2018
4,20008,10,10,4302.0,nan,1,600,3,2015,5,...,0,0,2015-05-14 17:50:46,2018-10-03 15:38:03,2015-05-14 18:00:00,nan,2015-05-14 18:42:23,2015-05-14 18:58:45,nan,2018
5,20010,12,12,12029.0,nan,1,600,3,2015,5,...,0,0,2015-05-14 17:53:46,2018-10-03 15:38:03,2015-05-14 18:43:15,nan,2015-05-14 20:13:06,2015-05-14 20:19:27,nan,2018
6,20012,14,14,3081.0,nan,1,600,4,2015,5,...,0,0,2015-05-15 12:51:26,2018-10-03 15:38:03,2015-05-15 13:21:29,nan,2015-05-15 14:07:18,2015-05-15 14:28:20,nan,2018
7,20014,16,16,nan,nan,1,200,4,2015,5,...,0,0,2015-05-15 13:17:49,2015-10-07 17:11:52,nan,nan,nan,nan,2015-05-15 14:06:21,2015
8,20016,18,18,3777.0,nan,1,600,4,2015,5,...,0,0,2015-05-15 19:30:16,2018-10-03 15:38:03,2015-05-15 19:45:03,nan,2015-05-15 20:53:19,2015-05-15 21:04:08,nan,2018
9,20018,20,20,12068.0,nan,1,300,5,2015,5,...,0,0,2015-05-16 14:49:45,2015-07-08 11:18:16,2015-05-16 16:08:45,nan,nan,nan,2015-05-16 17:21:43,2015
